In [7]:
import numpy as np
import pandas as pd
import geopandas as gpd
from torch.nn.functional import softmax
import torch

import datetime
import pickle

import ndpretty

import data_prep
from tqdm.notebook import tqdm

ndpretty.default()

In [2]:
data = data_prep.load_beijing_data()


Loaded air quality data from 87 devices. No weather data for []


In [3]:
devices = list(pd.read_table("Previous work/PM2.5-GNN/data/city_DS2.txt", delimiter=' ', header=None)[1])

In [4]:
df = data.measurements
df.index = df["time"]

In [5]:
R = np.empty((len(devices), len(devices)))
for idx_a, city_a in enumerate(tqdm(devices)):
    for idx_b, city_b in enumerate(tqdm(devices)):
        
        s_a = df[df["device_id"] == city_a]["pm25"]
        s_b = df[df["device_id"] == city_b]["pm25"]

        # corr = np.average(np.correlate(s_a, s_b))
        corr = pd.concat([s_a, s_b], axis=1).dropna().corr().iloc[0, 1]
        R[idx_a, idx_b] = corr

In [8]:
R_tensor = softmax(torch.tensor(R), dim=1)

In [9]:
R_tensor

12×12 torch.float64 tensor


interactive(children=(Text(value='[:, :]', description='Slice:', placeholder='e.g. [:, :]'), Output()), _dom_c…

In [76]:
torch.save(R_tensor, "R_corr_DS2.pt")